# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import math

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# View first 5 rows of table
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Calculate the total number of schools
number_of_schools = len(school_data_complete["school_name"].unique())

# Calculate the total number of students: use len(array) of student ID incase there are students with the same name
number_of_students = len(school_data_complete["Student ID"])

# Calculate the total budget: use sum of only the school data
total_budget = school_data["budget"].sum()

# Calculate the average math score
average_math_score = (school_data_complete["math_score"]).mean()

# Calculate the average math score
average_reading_score = (school_data_complete["reading_score"]).mean()

# Calculate the percentage of students with a passing math score (70 or greater)
passing_math = school_data_complete[school_data_complete["math_score"] >= 70]
percent_passing_math = (passing_math["math_score"].count())/number_of_students*100

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]
percent_passing_reading = (passing_reading["reading_score"].count())/number_of_students*100

# Calculate the overall passing rate (overall average score), i.e. (% passing math score + % passing reading)/2
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [3]:
# Create a dataframe to hold the above results
district_summary = {
    "Total Schools": [number_of_schools], 
    "Total Students": [number_of_students], 
    "Total Budget":[total_budget],
    "Average Math Score":[average_math_score],
    "Average Reading Score":[average_reading_score],
    "% Passing Math":[percent_passing_math],
    "% Passing Reading":[percent_passing_reading],
    "% Overall Passing Rate":[overall_passing_rate]
}
district_summary_df = pd.DataFrame(district_summary)

# Format displayed data
district_summary_df["Total Students"] = district_summary_df["Total Students"].apply('{:,.0f}'.format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].apply('${:,.2f}'.format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].apply('{:.2f}'.format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].apply('{:.2f}'.format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].apply('{:.2f}%'.format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].apply('{:.2f}%'.format)
district_summary_df["% Overall Passing Rate"] = district_summary_df["% Overall Passing Rate"].apply('{:.2f}%'.format)

# District Summary

In [4]:
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [6]:
# Rename columns for school name, type and budget, which will remain in school summary datafranme
school_data_complete.rename(columns={"school_name": "School Name", "type": "School Type", "budget": "Total School Budget", "Student ID": "Total Students"}, inplace=True)
school_data.rename(columns={"school_name": "School Name", "type": "School Type", "budget": "Total School Budget", "Student ID": "Total Students"}, inplace=True)

# Add column to apply conditional for checking if student is passing (>= 70) math and column for passing reading 
school_data_complete["passing_math"] = [0 if x < 70 else 1 for x in school_data_complete["math_score"]]
school_data_complete["passing_reading"] = [0 if x < 70 else 1 for x in school_data_complete["reading_score"]]

# Group by school
by_school = school_data_complete.set_index("School Name").groupby(["School Name"])

# School type
school_type = school_data.set_index("School Name")["School Type"]

# Total students per school
students_per_school = by_school["Total Students"].count()

# Total school budget
school_budget = school_data.set_index("School Name")["Total School Budget"]

# Per student budget
budget_per_student = school_budget/students_per_school

# Average math score by school
average_math_score = (by_school["math_score"]).mean()

# Average reading score by school 
average_reading_score = (by_school["reading_score"]).mean()

# Percent passing math score by school
percent_passing_math = (by_school["passing_math"].sum()/students_per_school)*100

# Percent passing reading score by school
percent_passing_reading = (by_school["passing_reading"].sum()/students_per_school)*100

# Overall passing rate (overall average score), i.e. (% passing math score + % passing reading)/2
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [7]:
# Create a dataframe to hold the above results
school_summary = pd.DataFrame({
    "School Type": school_type ,
    "Total Students": students_per_school,
    "Total School Budget": school_budget,
    "Per Student Budget": budget_per_student,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Math": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "% Overall Passing Rate": overall_passing_rate
})

# Arrange columns in order of display
school_summary_df = school_summary[[
    "School Type", 
    "Total Students", 
    "Total School Budget",
    "Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]]

# Format displayed data
school_summary_df["Total Students"] = school_summary_df["Total Students"].apply('{:,.0f}'.format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].apply('${:,.2f}'.format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].apply('${:,.2f}'.format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].apply('{:.2f}'.format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].apply('{:.2f}'.format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].apply('{:.2f}%'.format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].apply('{:.2f}%'.format)
school_summary_df["% Overall Passing Rate"] = school_summary_df["% Overall Passing Rate"].apply('{:.2f}%'.format)

# School Summary

In [8]:
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [9]:
school_summary_df.sort_values("% Overall Passing Rate", ascending = False)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%
Wright High School,Charter,"1,800","$1,049,400.00",$583.00,83.68,83.95,93.33%,96.61%,94.97%
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.36,83.73,93.87%,95.85%,94.86%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,94.38%
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,73.81%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [10]:
school_summary_df.sort_values("% Overall Passing Rate", ascending = True)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,94.38%
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.36,83.73,93.87%,95.85%,94.86%
Wright High School,Charter,"1,800","$1,049,400.00",$583.00,83.68,83.95,93.33%,96.61%,94.97%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [11]:
# Create variables that contain average math score by grade
student_data.rename(columns={"school_name": "School Name"}, inplace=True)
ninth = student_data.loc[student_data["grade"] == "9th"]
tenth = student_data.loc[student_data["grade"] == "10th"]
eleventh = student_data.loc[student_data["grade"] == "11th"]
twelfth = student_data.loc[student_data["grade"] == "12th"]

In [12]:
# Group by school name and average math score
ninth_math = ninth.groupby("School Name")["math_score"].mean()
tenth_math = tenth.groupby("School Name")["math_score"].mean()
eleventh_math = eleventh.groupby("School Name")["math_score"].mean()
twelfth_math = twelfth.groupby("School Name")["math_score"].mean()

In [13]:
# Create a dataframe to hold the above results
mathScores_byGrade = pd.DataFrame({
    "9th": ninth_math,
    "10th": tenth_math,
    "11th": eleventh_math,
    "12th": twelfth_math
})

# Arrange columns in order of display
mathScores_byGrade_df = mathScores_byGrade[[
    "9th", 
    "10th",
    "11th",
    "12th"
]]
mathScores_byGrade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [14]:
# Group by school name and average reading score
ninth_reading = ninth.groupby("School Name")["reading_score"].mean()
tenth_reading = tenth.groupby("School Name")["reading_score"].mean()
eleventh_reading = eleventh.groupby("School Name")["reading_score"].mean()
twelfth_reading = twelfth.groupby("School Name")["reading_score"].mean()

In [15]:
# Create a dataframe to hold the above results
readingScores_byGrade = pd.DataFrame({
    "9th": ninth_reading,
    "10th": tenth_reading,
    "11th": eleventh_reading,
    "12th": twelfth_reading
})

# Arrange columns in order of display
readingScores_byGrade_df = readingScores_byGrade[[
    "9th", 
    "10th",
    "11th",
    "12th"
]]
readingScores_byGrade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
# Bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_data_complete.columns

Index(['Total Students', 'student_name', 'gender', 'grade', 'School Name',
       'reading_score', 'math_score', 'School ID', 'School Type', 'size',
       'Total School Budget', 'passing_math', 'passing_reading'],
      dtype='object')

In [20]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student)
per_studentBudget = school_data_complete["Total School Budget"]/school_data_complete["size"]
school_data_complete["spending_bin"] = pd.cut(per_studentBudget, spending_bins, labels = group_names)

# Add column to apply conditional for checking if student is passing (>= 70) math and column for passing reading 
school_data_complete["passing_math"] = [0 if x < 70 else 1 for x in school_data_complete["math_score"]]
school_data_complete["passing_reading"] = [0 if x < 70 else 1 for x in school_data_complete["reading_score"]]

# Group by spending
by_spending = school_data_complete.groupby("spending_bin")

# Include Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, and Overall Passing Rate (Average of the above two

# Average math and reading scores
average_mathScore = by_spending["math_score"].mean()
average_readingScore = by_spending["reading_score"].mean()

# Percent passing math and reading
percent_passing_math = (by_spending["passing_math"].sum()/by_spending["School ID"].count())*100
percent_passing_reading = (by_spending["passing_reading"].sum()/by_spending["School ID"].count())*100

# Overall passing rate (overall average score), i.e. (% passing math score + % passing reading)/2
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [21]:
# Create a dataframe to hold the above results         
scores_by_spend = pd.DataFrame({
    "Average Math Score": average_mathScore,
    "Average Reading Score": average_readingScore,
    "% Passing Math": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "% Overall Passing Rate": overall_passing_rate
})
            
# Arrange columns in order of display
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]]

# Format displayed data
scores_by_spend.index.name = "Spending Ranges (Per Student)"
scores_by_spend["Average Math Score"] = scores_by_spend["Average Math Score"].apply('{:.2f}'.format)
scores_by_spend["Average Reading Score"] = scores_by_spend["Average Reading Score"].apply('{:.2f}'.format)
scores_by_spend["% Passing Math"] = scores_by_spend["% Passing Math"].apply('{:.2f}%'.format)
scores_by_spend["% Passing Reading"] = scores_by_spend["% Passing Reading"].apply('{:.2f}%'.format)
scores_by_spend["% Overall Passing Rate"] = scores_by_spend["% Overall Passing Rate"].apply('{:.2f}%'.format)
scores_by_spend

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.36,83.96,93.70%,96.69%,95.19%
$585-615,83.53,83.84,94.12%,95.89%,95.01%
$615-645,78.06,81.43,71.40%,83.61%,77.51%
$645-675,77.05,81.01,66.23%,81.11%,73.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [22]:
# Bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [23]:
# Create a table that breaks down school performances based on school size
school_data_complete["size_bins"] = pd.cut(school_data_complete["size"], size_bins, labels = group_names)

# Add column to apply conditional for checking if student is passing (>= 70) math and column for passing reading 
school_data_complete["passing_math"] = [0 if x < 70 else 1 for x in school_data_complete["math_score"]]
school_data_complete["passing_reading"] = [0 if x < 70 else 1 for x in school_data_complete["reading_score"]]

# Group by size
by_size = school_data_complete.groupby("size_bins")

# Include Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, and Overall Passing Rate (Average of the above two

# Average math and reading scores
average_mathScore = by_size["math_score"].mean()
average_readingScore = by_size["reading_score"].mean()

# Percent passing math and reading
percent_passing_math = (by_size["passing_math"].sum()/by_size["School ID"].count())*100
percent_passing_reading = (by_size["passing_reading"].sum()/by_size["School ID"].count())*100

# Overall passing rate (overall average score), i.e. (% passing math score + % passing reading)/2
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [24]:
# Create a dataframe to hold the above results
scores_by_size = pd.DataFrame({
    "Average Math Score": average_mathScore,
    "Average Reading Score": average_readingScore,
    "% Passing Math": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "% Overall Passing Rate": overall_passing_rate
})
            
# Arrange columns in order of display
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]]

# Format displayed data
scores_by_size.index.name = "School Size"
scores_by_size["Average Math Score"] = scores_by_size["Average Math Score"].apply('{:.2f}'.format)
scores_by_size["Average Reading Score"] = scores_by_size["Average Reading Score"].apply('{:.2f}'.format)
scores_by_size["% Passing Math"] = scores_by_size["% Passing Math"].apply('{:.2f}%'.format)
scores_by_size["% Passing Reading"] = scores_by_size["% Passing Reading"].apply('{:.2f}%'.format)
scores_by_size["% Overall Passing Rate"] = scores_by_size["% Overall Passing Rate"].apply('{:.2f}%'.format)
scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.83,83.97,93.95%,96.04%,95.00%
Medium (1000-2000),83.37,83.87,93.62%,96.77%,95.19%
Large (2000-5000),77.48,81.20,68.65%,82.13%,75.39%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [26]:
# Add column to apply conditional for checking if student is passing (>= 70) math and column for passing reading 
school_data_complete["passing_math"] = [0 if x < 70 else 1 for x in school_data_complete["math_score"]]
school_data_complete["passing_reading"] = [0 if x < 70 else 1 for x in school_data_complete["reading_score"]]

# Group by school type
by_type = school_data_complete.groupby("School Type")

# Include Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, and Overall Passing Rate (Average of the above two

# Average math and reading scores
average_mathScore = by_type["math_score"].mean()
average_readingScore = by_type["reading_score"].mean()

# Percent passing math and reading
percent_passing_math = (by_type["passing_math"].sum()/by_type["School ID"].count())*100
percent_passing_reading = (by_type["passing_reading"].sum()/by_type["School ID"].count())*100

# Overall passing rate (overall average score), i.e. (% passing math score + % passing reading)/2
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [27]:
# Create a dataframe to hold the above results
scores_by_type = pd.DataFrame({
    "Average Math Score": average_mathScore,
    "Average Reading Score": average_readingScore,
    "% Passing Math": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "% Overall Passing Rate": overall_passing_rate
})
            
# Arrange columns in order of display
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]]

# Format displayed data
scores_by_type.index.name = "School Type"
scores_by_type["Average Math Score"] = scores_by_type["Average Math Score"].apply('{:.2f}'.format)
scores_by_type["Average Reading Score"] = scores_by_type["Average Reading Score"].apply('{:.2f}'.format)
scores_by_type["% Passing Math"] = scores_by_type["% Passing Math"].apply('{:.2f}%'.format)
scores_by_type["% Passing Reading"] = scores_by_type["% Passing Reading"].apply('{:.2f}%'.format)
scores_by_type["% Overall Passing Rate"] = scores_by_type["% Overall Passing Rate"].apply('{:.2f}%'.format)
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%
